In [1]:
import pandas as pd
import pickle
import numpy as np
import time 

time: 161 ms (started: 2022-05-20 16:33:05 +00:00)


In [2]:
def map_feature_label(target_path,dict_csv_path):
    all_feature=pd.read_csv(dict_csv_path)
    feature_label_divtionary={k:v for k,v in zip(all_feature['feature_name'],all_feature['label'])}
    item_feature=pd.read_csv(target_path)
    item_feature['feature_category_id'] = item_feature['feature_category_id'].apply(str)
    item_feature['feature_value_id'] = item_feature['feature_value_id'].apply(str)
    item_feature['feature_name']=item_feature["feature_category_id"] + "_" + item_feature["feature_value_id"]
    item_feature['label']=[feature_label_divtionary[i] for i in item_feature['feature_name']]
    item_feature['label'].apply(int)
    return item_feature

time: 555 µs (started: 2022-05-20 16:33:05 +00:00)


# Data preprocessing
- item=>vector
- session=>vector

## item==>vetor
### 1. mapping

In [3]:
item_feature=map_feature_label('../../recsys2022/item_features.csv','all_feature_encoding.csv')
item_feature["frequence"]=np.ones((len(item_feature),1))
item_feature

,item_id,feature_category_id,feature_value_id,feature_name,label,frequence
0,2,56,365,56_365,637,1.0
1,2,62,801,62_801,720,1.0
2,2,68,351,68_351,796,1.0
3,2,33,802,33_802,378,1.0
4,2,72,75,72_75,878,1.0
...,...,...,...,...,...,...
471746,28143,68,351,68_351,796,1.0
471747,28143,55,390,55_390,578,1.0
471748,28143,11,109,11_109,131,1.0
471749,28143,73,91,73_91,902,1.0


time: 433 ms (started: 2022-05-20 16:33:05 +00:00)


In [4]:
#pivot table
item_feature_p = item_feature.pivot_table(index="item_id", columns="feature_name",values="frequence")
item_feature_p = (item_feature_p.fillna(0))
item_feature_p.head()

feature_name,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
item_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 385 ms (started: 2022-05-20 16:33:06 +00:00)


## candidate_items => vector

In [5]:
def items2vec(_df_item,_session_items):
    session_vec = np.sum(_df_item.loc[pd.Index(_session_items)]).values
    return session_vec 

time: 317 µs (started: 2022-05-20 16:33:06 +00:00)


In [6]:
item_feature_reset_ind = item_feature_p.reset_index()
candidate_raw = pd.read_csv("../../recsys2022/candidate_items.csv")
candidate_vector = pd.merge(candidate_raw, item_feature_reset_ind, how='left', on="item_id")
candidate_vector.to_csv("candidate_vector.csv", index=False)

time: 1.12 s (started: 2022-05-20 16:33:06 +00:00)
